In [1]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from utilities.data_manager import DataManager
from utilities.backtest_analysis import BacktestAnalysis
from strategies import envelope as strat

In [3]:
data = DataManager(name="binance")  # "bitget"

In [4]:
symbol = "BTC/USDT" #"BTC/USDT:USDT"
ohlcv = data.load(symbol, timeframe="1d", start_date="2025-01-01") #start_date="2024-01-01 00:00:00")
strategy_params = {
    'average_type': 'SMA',  # 'SMA', 'EMA', 'WMA', 'DCM'
    'average_period': 6,
    'envelopes': [0.07, 0.11, 0.14],
    'stop_loss_pct': 0.3,
    # 'price_jump_pct': 0.3,
    #'position_size_percentage': 100,  #  % of the balance spread equally across each envelope
     'position_size_fixed_amount': 1000,  # fixed amount spread equally across each envelope
    # 'mode': "long", "short", 'both' (default)
}

# symbol = "ETH/USDT:USDT"
# ohlcv = data.load(symbol, timeframe="1h", start_date="2021-01-01 00:00:00")
# strategy_params = {
#     'average_type': 'DCM',
#     'average_period': 5,
#     'envelopes': [0.07, 0.11, 0.14],
#     'stop_loss_pct': 0.4,
#     'position_size_percentage': 100,
# }

# symbol = "ADA/USDT:USDT"
# ohlcv = data.load(symbol, timeframe="1h", start_date="2021-01-01 00:00:00")
# strategy_params = {
#     'average_type': 'DCM',
#     'average_period': 5,
#     'envelopes': [0.07, 0.11, 0.14, 0.17],
#     'stop_loss_pct': 0.4,
#     'price_jump_pct': 0.3,
#     'position_size_percentage': 100,
# }

# symbol = "AVAX/USDT:USDT"
# ohlcv = data.load(symbol, timeframe="1h", start_date="2021-01-01 00:00:00")
# strategy_params = {
#     'average_type': 'SMA',
#     'average_period': 5,
#     'envelopes': [0.1, 0.15, 0.2, 0.25],
#     'stop_loss_pct': 0.4,
#     'price_jump_pct': 0.3,
#     'position_size_percentage': 100,
# }

# symbol = "SOL/USDT:USDT"
# ohlcv = data.load(symbol, timeframe="1h", start_date="2021-01-01 00:00:00")
# strategy_params = {
#     'average_type': 'SMA',
#     'average_period': 5,
#     'envelopes': [0.07, 0.11, 0.14, 0.17],
#     'stop_loss_pct': 0.4,
#     # 'price_jump_pct': 0.3,
#     'position_size_percentage': 100,
# }

# symbol = "LINK/USDT:USDT"
# ohlcv = data.load(symbol, timeframe="1h")
# strategy_params = {
#     'average_type': 'SMA',
#     'average_period': 5,
#     'envelopes': [0.1, 0.15, 0.2, 0.25],
#     'stop_loss_pct': 0.4,
#     'price_jump_pct': 0.2,
#     'position_size_percentage': 100,
# }

strategy = strat.Strategy(strategy_params, ohlcv)
strategy.run_backtest(initial_balance=1000, leverage=1, open_fee_rate=0.0002, close_fee_rate=0.0006)

In [5]:
results = BacktestAnalysis(strategy)
results.print_metrics()
path = f"/home/spoonbill/Projects/RobotTraders/CryptoStrategyLab/code/results/BTCUSD"
results.save_metrics_as_csv(path)

--- General ---
Period: [2025-01-01 00:00:00] -> [2025-04-07 00:00:00]
Initial balance: 1000.0 $
Final balance: 1098.26 $
Performance: 9.83 %
Hodl performance: -16.31%
Performance/Hodl: 31.23 %
Total trades: 7
Time in position: 20.83 %

--- Health ---
Win rate: 85.71 %
Max drawdown at trade close: -0.06 %
Max drawdown at equity update: -2.71 %
Profit factor: 136.69
Return over max drawdown: 3.63
Sharpe ratio: 3.55
Sortino ratio: 3.87
Calmar ratio: 13.35

--- Trades ---
Average net PnL: 3.96 %
Average trades per day: 0.072
Average trades duration: 2 days 20:34:17.142857142
Best trade: +7.35 % entered 2025-03-03 00:00:00 exited 2025-03-04 00:00:00
Worst trade: -0.1 % entered 2025-02-25 00:00:00 exited 2025-03-02 00:00:00
Total winning trades: 6
Total loosing trades: 1
Average net PnL winning trades: 4.63 %
Average net PnL loosing trades: -0.1 %
Mean winning trades duration: 2 days 12:00:00
Mean loosing trades duration: 5 days 00:00:00
Max win streak: 4
Max lose streak: 1
Open reasons:
Op

AttributeError: 'str' object has no attribute 'strategy_id'

In [ ]:
strategy.trades_info
# path = symbol.replace('/', '-').replace(':', '-') 
# strategy.save_trades_info(path)

In [ ]:
strategy.equity_record
# strategy.save_equity_record(path)

In [ ]:
results.plot_equity()
# results.plot_equity(path)

results.plot_drawdown()
# results.plot_drawdown(path)

In [ ]:
results.plot_monthly_performance(year="all")
# results.plot_monthly_performance(year="all", path)

In [ ]:
indicators = {
    'average': {
        'color': "white",
        'df': pd.DataFrame({
            'time': strategy.data.index,
            'average': strategy.data['average'],
        }).dropna()        
    } 
}

for i, e in enumerate(strategy_params['envelopes']):
    indicators.update({
        f'band_high_{i+1}' : {
            'color': "red",
            'df': pd.DataFrame({
                'time': strategy.data.index,
                f'band_high_{i+1}': strategy.data[f'band_high_{i+1}'],
            }).dropna()   
        },
        f'band_low_{i+1}' : {
            'color': "green",
            'df': pd.DataFrame({
                'time': strategy.data.index,
                f'band_low_{i+1}': strategy.data[f'band_low_{i+1}'],
            }).dropna()  
        },
    })

results.plot_candlestick(indicators=indicators)